In [1]:
from skimage import io
import pandas as pd
import os
from shutil import copyfile
from skimage import segmentation
from skimage.morphology import watershed
from skimage import morphology
from skimage.feature import peak_local_max
from scipy import ndimage
import numpy as np
import scipy
import matplotlib.pyplot as plt
import cv2

df = pd.DataFrame(columns=('name', 'x', 'y', 'bpc', 'blobs', 'lines', 's1', 's2', 's3', 's4'))

'''
compare_list = []
compare_list.append(('input/4503.png', 6)) #U
compare_list.append(('input/39.png' , 1)) #r
compare_list.append(('input/201.png', 2)) #i
compare_list.append(('input/823.png', 3)) #t
compare_list.append(('input/201.png', 4)) #i
compare_list.append(('input/3860.png', 5)) #M
compare_list.append(('input/4503.png', 6)) #U
compare_list.append(('input/2150.png', 7)) #C
compare_list.append(('input/1154.png', 7)) #Ż
compare_list.append(('input/6076.png', 8)) #E
compare_list.append(('input/5002.png', 9)) #R
'''

def get_max_hough_similarity(image1, image2):
    image1 = cv2.GaussianBlur(image1,(3,3),0)
    image2 = cv2.GaussianBlur(image2,(3,3),0)
    sim    = 0
    simsum = 0
    maxs = 0
    x1, y1, _ = image1.shape
    x2, y2, _ = image2.shape
    dx = 0
    dy = 0 
    for dx in range(-3,4):
        for dy in range(-3, 4):
            sim = 0
            simsum = 0
            for i in range(0, min(x1,x2)):
                for j in range(0, min(y1,y2)):
                    if i+dx >= 0 and i+dx < x2 and j+dy >= 0 and j+dy < y2:
                        if image1[i][j][0] == image2[i+dx][j+dy][0]:
                            sim = sim + 1
                        simsum = simsum + 1    
            #print 'sim %d simsum %d' % (sim, simsum)
            if simsum != 0:
                maxs = max(maxs, (1.0*sim/simsum) * 100)
    #print 'dx=%d dy=%d %d - %d = %d p' % (dx, dy, sim, simsum, (1.0*sim/simsum * 100))
    return maxs

def iterate_hough(image1):
    ans = 0
    for path, val in compare_list:
        #print path
        image2 = io.imread(path)
        value = get_max_hough_similarity(image1, image2)
        if (value > 85):
            ans += val * 100000000
    return ans

def get_intensity(image):
    bpc = 0
    low_bc = 0
    high_bc = 0
    left_bc = 0
    right_bc = 0
 
    mx, my, _ = image.shape

    for iy in range(my):
        for ix in range(mx):
            if image[ix][iy][0] == 0:
                bpc = bpc + 1
                if iy < (my/2):
                    left_bc = left_bc + 1
                if iy >= (my/2):
                    right_bc = right_bc + 1
                if ix < (mx/2):
                    low_bc = low_bc + 1
                if ix >= (mx/2):
                    high_bc = high_bc + 1
    return (bpc, low_bc, high_bc, left_bc, right_bc)

def get_lines(img):
    img = ndimage.minimum_filter(img, size=2)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,50,150,apertureSize = 3)
    lines = cv2.HoughLines(edges,1,np.pi/180,10)
    if lines is None:
        num_lin = 0
    else:
        num_lin = len(lines[0])
    
    lines = cv2.HoughLines(edges,1,np.pi/180,10)
    if lines is None:
        num_lin = 0
    else:
        num_lin = len(lines[0])
    return num_lin

def get_blobs(image):
    im = ndimage.minimum_filter(image, size=2)
    blobs, number_of_blobs = ndimage.label(im)
    return number_of_blobs
    

In [2]:
for i in range(1, 6169):
    image_path = 'input/%d.png' % i
    image_name = '%d.png' % i
    image = io.imread(image_path)    
    img   = cv2.imread(image_path)
    x,y,_ = image.shape    
    
    bpc, lwbpc, hbpc, ltbpc, rbpc = get_intensity(image)
    num_lin = get_lines(img)
    number_of_blobs = get_blobs(image)
    #simil = iterate_hough(image)

    df.loc[i] = [image_name, int(x),int(y),int(bpc), 1000 * int(number_of_blobs), 5 * int(num_lin),
                 int(lwbpc), int (hbpc), int(ltbpc), int(rbpc)]
    if (i%100 == 0):
        print 'Progress %d / 6168' % i
print 'Calculated'

Progress 100 / 6168
Progress 200 / 6168
Progress 300 / 6168
Progress 400 / 6168
Progress 500 / 6168
Progress 600 / 6168
Progress 700 / 6168
Progress 800 / 6168
Progress 900 / 6168
Progress 1000 / 6168
Progress 1100 / 6168
Progress 1200 / 6168
Progress 1300 / 6168
Progress 1400 / 6168
Progress 1500 / 6168
Progress 1600 / 6168
Progress 1700 / 6168
Progress 1800 / 6168
Progress 1900 / 6168
Progress 2000 / 6168
Progress 2100 / 6168
Progress 2200 / 6168
Progress 2300 / 6168
Progress 2400 / 6168
Progress 2500 / 6168
Progress 2600 / 6168
Progress 2700 / 6168
Progress 2800 / 6168
Progress 2900 / 6168
Progress 3000 / 6168
Progress 3100 / 6168
Progress 3200 / 6168
Progress 3300 / 6168
Progress 3400 / 6168
Progress 3500 / 6168
Progress 3600 / 6168
Progress 3700 / 6168
Progress 3800 / 6168
Progress 3900 / 6168
Progress 4000 / 6168
Progress 4100 / 6168
Progress 4200 / 6168
Progress 4300 / 6168
Progress 4400 / 6168
Progress 4500 / 6168
Progress 4600 / 6168
Progress 4700 / 6168
Progress 4800 / 6168
P

In [3]:
def convert(x):
    try:
        return x.astype(int)
    except:
        return x
df = df.apply(convert)
print 'converted to df, starting aggregation...'

converted to df, starting aggregation...


In [4]:
from sklearn.cluster import AgglomerativeClustering
cl = AgglomerativeClustering(n_clusters=59)
df2 = cl.fit_predict(df.drop('name', axis=1))
print 'Agregated, starting saving'

Agregated, starting saving


In [5]:
s = pd.Series(df2, index=range(1,len(df2)+1))
df3 = pd.concat([df, s], axis=1, ignore_index=True)

In [6]:
path = "category"
os.mkdir(path)
for i in range(0,60):
    path = "category/%d" % i
    os.mkdir(path, 0755 );

for r in range(1,6169):
    image_name = df3.loc[r][0]
    image_cat = df3.loc[r][df3.shape[1] - 1]
    image_path_in = 'input/%s' % image_name
    image_path_out = 'category/%d/%s' % (image_cat, image_name)
    copyfile(image_path_in, image_path_out)
    if r % 500 == 0:
        print image_path_in + " " + image_path_out
print 'Saved.'

input/500.png category/3/500.png
input/1000.png category/4/1000.png
input/1500.png category/5/1500.png
input/2000.png category/14/2000.png
input/2500.png category/9/2500.png
input/3000.png category/4/3000.png
input/3500.png category/4/3500.png
input/4000.png category/1/4000.png
input/4500.png category/5/4500.png
input/5000.png category/14/5000.png
input/5500.png category/6/5500.png
input/6000.png category/2/6000.png
Saved.


In [7]:
print 'Prepare submission'
df3.to_csv('submission.csv', index=False, header=False)
print 'Submission saved.'

Prepare submission
Submission saved.


In [8]:
'''
df3.loc[2912]
for i in range(1, 2):
    image_path = 'input/%d.png' % i
    image_name = '%d.png' % i
    image = io.imread(image_path)
    distance = ndimage.distance_transform_edt(image)
    local_maxi = peak_local_max(image, indices=False)
    markers = morphology.label(local_maxi)
    labels_ws = watershed(-distance, markers, mask=image)
    #markers[~image] = -1
    #labels_rw = segmentation.random_walker(image, markers)
labels_ws.shape

import cv2
import numpy as np


#flatten to make greyscale, using your second red-black image as input.
#img = scipy.misc.imread('input/903.png',flatten=1)
#smooth and threshold as image has compression artifacts (jpg)

im[im<10]=0
img = cv2.imread('input/87.png')
#img = ndimage.minimum_filter(img, size=2)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
lines = cv2.HoughCircles(gray,cv2.cv.CV_HOUGH_GRADIENT,1,300,50,10)
if lines is None:
    num_lin = -1
else:
    num_lin = len(lines)
"""
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 100*(-b))
    y1 = int(y0 + 100*(a))
    x2 = int(x0 - 100*(-b))
    y2 = int(y0 - 100*(a))
    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),1)"""

#plt.imshow(edges)
#plt.show()
print num_lin
#cv2.imwrite('houghlines3.jpg',img)



SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-8-0d4eb2191bfc>, line 48)

In [ ]:
df3.loc[4503]

In [10]:
def kaze_match(im1_path, im2_path):
    # load the image and convert it to grayscale
    im1 = cv2.imread(im1_path)
    im2 = cv2.imread(im2_path)
    gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)    

    # initialize the AKAZE descriptor, then detect keypoints and extract
    # local invariant descriptors from the image
    detector = cv2.AKAZE_create()
    (kps1, descs1) = detector.detectAndCompute(gray1, None)
    (kps2, descs2) = detector.detectAndCompute(gray2, None)

    print("keypoints: {}, descriptors: {}".format(len(kps1), descs1.shape))
    print("keypoints: {}, descriptors: {}".format(len(kps2), descs2.shape))    

    # Match the features
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.knnMatch(descs1,descs1, k=2)

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.9*n.distance:
            good.append([m])

    # cv2.drawMatchesKnn expects list of lists as matches.
    im3 = cv2.drawMatchesKnn(im1, kps1, im2, kps2, good[1:20], None, flags=2)
    cv2.imshow("AKAZE matching", im3)
    cv2.waitKey(0) 

kaze_match('input/1154.png', 'input/1632.png')

AttributeError: 'module' object has no attribute 'AKAZE_create'